In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using JSON
using Dates
using TimeZones
using Chain
using ResultTypes

using Base: @kwdef

In [ ]:
using Superfacility: SFAPI

Grab a token and check if it works

In [ ]:
client = SFAPI.Token.Client(joinpath(@__DIR__, "credentials"))
tc = SFAPI.Token.fetch(client) |> unwrap;

In [ ]:
tc = SFAPI.Token.refresh(tc)
account = SFAPI.Account.User(SFAPI.Query.get("account", tc.token))

Paths at NERSC:

In [ ]:
home_path = "/global/homes/$(account.name[1])/$(account.name)"
scratch_path = "/pscratch/sd/$(account.name[1])/$(account.name)"

Start a command (executable) to list a directory:

In [ ]:
tc = SFAPI.Token.refresh(tc)
cmd = SFAPI.Executable.run("ls $(home_path)", tc)

This is a handle to a command that just ran ("OK"), now grab the result from the command. Note that the `result` function returns a Julia task => you need to use `fetch` to get the result.

In [ ]:
t = SFAPI.Executable.result(cmd, tc)
fetch(t)

Now, lets try something a bit more involved, by running a Bash script

In [ ]:
cmd_str = "sleep 10\necho hi"
println(cmd_str)

In [ ]:
tc = SFAPI.Token.refresh(tc)
cmd = SFAPI.Executable.run( 
    "cat << EOF | bash\n$(cmd_str)\nEOF" ,
    tc
)

This is how you would repeatedly poll the SFAPI to check if the command is done running. `SFAPI.Executable.result` does repeatedly check the status, and fetches the result when done (continuously refreshing the API token). So progress is made even when not checking `istaskdone`

In [ ]:
t = SFAPI.Executable.result(cmd, tc)
while ! istaskdone(t)
    println("Wating for result ...")
    sleep(1)
end
println(fetch(t))